# Сегментация

In [ ]:
#!pip install pooch

In [ ]:
import skimage
from skimage import data, restoration, util, img_as_float, filters, color, morphology, segmentation
from skimage.filters import threshold_otsu, try_all_threshold, threshold_niblack, threshold_sauvola, threshold_multiotsu

from skimage.filters import sobel, rank
from skimage.measure import label
from skimage.color import label2rgb

from skimage.segmentation import (chan_vese, morphological_chan_vese,
                                  morphological_geodesic_active_contour,
                                  inverse_gaussian_gradient,
                                  checkerboard_level_set)

from skimage.segmentation import watershed, expand_labels, flood, flood_fill
from skimage.feature import peak_local_max
from skimage.transform import hough_line, hough_line_peaks, probabilistic_hough_line
 
from skimage.feature import canny
from skimage.draw import line
from skimage.util import img_as_ubyte

from skimage.graph import RAG
from skimage import graph

from skimage.segmentation import random_walker
from skimage.data import binary_blobs
from skimage.exposure import rescale_intensity

from skimage.morphology import erosion, dilation, opening, closing, white_tophat, black_tophat, square
from skimage.morphology import skeletonize, convex_hull_image, medial_axis, thin
from skimage.morphology import disk

from skimage.draw import ellipse
from skimage.measure import label, regionprops, regionprops_table
from skimage.transform import rotate

from skimage.metrics import (adapted_rand_error,
                              variation_of_information)
from skimage.filters import sobel
from skimage.measure import label
from skimage.util import img_as_float
from skimage.feature import canny
from skimage.morphology import remove_small_objects
from skimage.segmentation import (morphological_geodesic_active_contour,
                                  inverse_gaussian_gradient,
                                  watershed,
                                  mark_boundaries)

from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier
from functools import partial

import networkx as nx
import imageio
import imutils
import cv2

from matplotlib import cm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from collections import deque
from math import sqrt, exp
import numpy as np
import scipy.ndimage as ndi
import pandas as pd
import math

In [ ]:
def show_binary(image, binary):
    fig, axes = plt.subplots(ncols=3, figsize=(16, 4))
    ax = axes.ravel()
    ax[0] = plt.subplot(1, 3, 1)
    ax[1] = plt.subplot(1, 3, 2)
    ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

    ax[0].imshow(image, cmap=plt.cm.gray)
    ax[0].set_title('Original')
    ax[0].axis('off')

    ax[1].hist(image.ravel(), bins=256)
    ax[1].set_title('Histogram')
    ax[1].axvline(thresh, color='r')

    ax[2].imshow(binary, cmap=plt.cm.gray)
    ax[2].set_title('Thresholded')
    ax[2].axis('off')

    plt.show()

## 1. Пороговая сегментация
## 1.1. Базовая сегментация на основе бинаризации по глобальному яркостному порогу
__Метод Оцу__ вычисляет «оптимальный» порог (отмечен красной линией на гистограмме ниже) путем максимизации дисперсии между двумя классами пикселей, которые разделены пороговым значением. Равным образом этот порог минимизирует внутриклассовую дисперсию.

In [ ]:
image = data.coins()
thresh = threshold_otsu(image)
binary = image > thresh

show_binary(image, binary)

Негативное влияние неравномерной освещенности на результат сегментации по порогу.

In [ ]:
fig, ax = try_all_threshold(image, figsize=(10, 8), verbose=False)
plt.show()

## 1.2. Устранение неравномерности освещения
__Алгоритм катящегося мяча для оценки интенсивности фона__.
Алгоритм катящегося мяча оценивает интенсивность фона изображения в оттенках серого в случае неравномерной экспозиции. Он часто используется в биомедицинской обработке изображений и был впервые предложен Стэнли Р. Стернбергом в 1983 году. 

Алгоритм работает как фильтр и интуитивно понятен. Мы представляем изображение как поверхность, на которой блоки единичного размера наложены друг на друга вместо каждого пикселя. Количество блоков и, следовательно, высота поверхности определяется интенсивностью пикселя. Чтобы получить интенсивность фона в желаемом (пиксельном) положении, мы представляем погружение шара под поверхность в желаемом месте. Когда мяч полностью покрыт блоками, вершина шара определяет интенсивность фона в этой позиции. Затем мы можем катать этот шар под поверхностью, чтобы получить значения фона для всего изображения.

Scikit-image реализует обобщенную версию этого алгоритма, который позволяет вам использовать произвольные формы в качестве ядра и работает с n-мерными изображениями. Это позволяет напрямую фильтровать изображения RGB или фильтровать стеки изображений по любому (или всем) пространственным измерениям.

In [ ]:
background = restoration.rolling_ball(image)
image_result = image - background
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))

ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original image')
ax[0].axis('off')

ax[1].imshow(background, cmap='gray')
ax[1].set_title('Background')
ax[1].axis('off')

ax[2].imshow(image_result, cmap='gray')
ax[2].set_title('Result')
ax[2].axis('off')

fig.tight_layout()
plt.show()

In [ ]:
thresh = threshold_otsu(image_result)
binary = image_result > thresh

show_binary(image_result, binary)

## 1.3. Локальные методы бинаризации
Для учета эффектов неравномерности освещенности предлагается применять специализированные методы бинаризации. __Пороги Ниблака и Саувола__ - это локальные методы определения пороговых значений, которые полезны для изображений с неоднородным фоном, особенно при распознавании текста. Вместо расчета единого глобального порога для всего изображения, несколько пороговых значений вычисляются для каждого пикселя с использованием определенных формул, которые учитывают среднее значение и стандартное отклонение локальной окрестности (определяемой окном с центром вокруг пикселя).

In [ ]:
image = data.page()
binary_global = image > threshold_otsu(image)

window_size = 25
thresh_niblack = threshold_niblack(image, window_size=window_size, k=0.8)
thresh_sauvola = threshold_sauvola(image, window_size=window_size)

binary_niblack = image > thresh_niblack
binary_sauvola = image > thresh_sauvola

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 1)
plt.imshow(image, cmap=plt.cm.gray)
plt.title('Original')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.title('Global Threshold')
plt.imshow(binary_global, cmap=plt.cm.gray)
plt.axis('off')

plt.subplot(2, 2, 3)
plt.imshow(binary_niblack, cmap=plt.cm.gray)
plt.title('Niblack Threshold')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(binary_sauvola, cmap=plt.cm.gray)
plt.title('Sauvola Threshold')
plt.axis('off')

plt.show()

## 1.4. Обработка с несколькими порогами
Множественные пороги Оцу - это алгоритм пороговой обработки, который используется для разделения пикселей входного изображения на несколько различных классов, каждый из которых получается в соответствии с интенсивностью уровней серого в изображении.

Multi-Otsu рассчитывает несколько пороговых значений, определяемых количеством желаемых классов. По умолчанию количество классов равно 3: для получения трех классов алгоритм возвращает два пороговых значения. Они представлены красными линиями на гистограмме ниже. 

In [ ]:
image = data.camera()

# Applying multi-Otsu threshold for the default value, generating
# three classes.
thresholds = threshold_multiotsu(image)

# Using the threshold values, we generate the three regions.
regions = np.digitize(image, bins=thresholds)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Plotting the original image.
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original')
ax[0].axis('off')

# Plotting the histogram and the two thresholds obtained from
# multi-Otsu.
ax[1].hist(image.ravel(), bins=255)
ax[1].set_title('Histogram')
for thresh in thresholds:
    ax[1].axvline(thresh, color='r')

# Plotting the Multi Otsu result.
ax[2].imshow(regions, cmap='jet')
ax[2].set_title('Multi-Otsu result')
ax[2].axis('off')

plt.subplots_adjust()

plt.show()

In [ ]:
man_mask = np.int8(regions == 0)
plt.imshow(man_mask, cmap="gray")
plt.show()

In [ ]:
selem = disk(6)
opened = opening(man_mask, selem)

plt.imshow(opened, cmap="gray")
plt.show()

In [ ]:
opened_mask = opened == 1
hull2 = convex_hull_image(opened_mask)
hull_diff = img_as_float(hull2.copy())
hull_diff[opened] = 1
resulting = hull_diff * image

fig, ax = plt.subplots()
ax.imshow(resulting, cmap=plt.cm.gray)
ax.set_title('Object detection')
plt.show()

## 2. Установление границ на основе минимизации энергии
__Алгоритм сегментации Чан-Весе__ предназначен для сегментирования объектов без четко определенных границ. Этот алгоритм основан на наборах уровней, которые итеративно изменяются для минимизации энергии, которая определяется взвешенными значениями, соответствующими сумме различий интенсивности от среднего значения за пределами сегментированной области, сумме отличий от среднего значения внутри сегментированной области.

Этот алгоритм был впервые предложен Тони Чаном и Люминитой Весе. Предлагаемая реализация алгоритма в библиотеке Scikit-image подходит только для изображений в градациях серого.

Типичные значения лямбда1 и лямбда2 равны 1. Если «фон» сильно отличается от сегментированного объекта с точки зрения распределения (например, однородное черное изображение с фигурами разной интенсивности), то эти значения должны отличаться друг от друга.

Типичные значения для mu находятся в диапазоне от 0 до 1, хотя более высокие значения могут использоваться при работе с формами с очень плохо очерченными контурами.

Алгоритм также возвращает список значений, соответствующих энергии на каждой итерации. Это можно использовать для настройки различных параметров, описанных выше.

In [ ]:
image = img_as_float(data.camera())
cv = chan_vese(image, mu=0.25, lambda1=1, lambda2=1, tol=1e-3, max_num_iter=200,
               dt=0.5, init_level_set="checkerboard", extended_output=True)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8))
ax = axes.flatten()

ax[0].imshow(image, cmap="gray")
ax[0].set_axis_off()
ax[0].set_title("Original Image", fontsize=12)

ax[1].imshow(cv[0], cmap="gray")
ax[1].set_axis_off()
title = "Chan-Vese segmentation - {} iterations".format(len(cv[2]))
ax[1].set_title(title, fontsize=12)

ax[2].imshow(cv[1], cmap="gray")
ax[2].set_axis_off()
ax[2].set_title("Final Level Set", fontsize=12)

ax[3].plot(cv[2])
ax[3].set_title("Evolution of energy over iterations", fontsize=12)

fig.tight_layout()
plt.show()

In [ ]:
cv[1].shape
thresh = threshold_otsu(cv[1])
binary = cv[1] > thresh

show_binary(cv[1], binary)

## 3. Сегментирование на основе контуров и границ
## 3.1. Поиск прямых линий на основе преобразования Хафа
Преобразование Хафа в его простейшей форме - это метод обнаружения прямых.

В следующем примере мы создаем изображение с пересечением линии. Затем мы используем преобразование Хафа, чтобы исследовать пространство параметров для прямых линий, которые могут проходить через изображение.

Обычно линии параметризуются как y = mx + c с градиентом m и точкой пересечения y c. Однако это означало бы, что m стремится к бесконечности для вертикальных линий. Вместо этого мы строим отрезок, перпендикулярный линии, ведущий в начало координат. Линия представлена длиной этого сегмента r и углом, который он составляет с осью x, θ.

Преобразование Хафа создает массив гистограмм, представляющий пространство параметров (то есть матрицу M × N, для M различных значений радиуса и N различных значений θ). Для каждой комбинации параметров, r и θ, мы затем находим количество ненулевых пикселей во входном изображении, которые будут располагаться близко к соответствующей строке, и соответствующим образом увеличиваем массив в позиции (r, θ).

Локальные максимумы на полученной гистограмме указывают параметры наиболее вероятных линий. В нашем примере максимумы наблюдаются под углом 45 и 135 градусов, что соответствует углам вектора нормали каждой линии.

Другой подход - это прогрессивное вероятностное преобразование Хафа. Такой подход основан на предположении, что использование случайного подмножества точек голосования дает хорошее приближение к фактическому результату, и что линии могут быть извлечены во время процесса голосования путем обхода связанных компонентов. Это возвращает начало и конец каждого сегмента линии.

Функция probabilistic_hough имеет три параметра: общий порог, который применяется к аккумулятору Хафа, минимальная длина строки и промежуток между строками, который влияет на объединение строк. В приведенном ниже примере мы находим строки длиной более 10 пикселей с зазором менее 3 пикселей.

In [ ]:
# Constructing test image
image = np.zeros((200, 200))
idx = np.arange(25, 175)
image[idx, idx] = 255
image[line(45, 25, 25, 175)] = 255
image[line(25, 135, 175, 155)] = 255

# Classic straight-line Hough transform
# Set a precision of 0.5 degree.
tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)
h, theta, d = hough_line(image, theta=tested_angles)

In [ ]:
# Generating figure 1
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')
ax[0].set_axis_off()

angle_step = 0.5 * np.diff(theta).mean()
d_step = 0.5 * np.diff(d).mean()
bounds = [np.rad2deg(theta[0] - angle_step),
          np.rad2deg(theta[-1] + angle_step),
          d[-1] + d_step, d[0] - d_step]
ax[1].imshow(np.log(1 + h), extent=bounds, cmap=cm.gray, aspect=1 / 1.5)
ax[1].set_title('Hough transform')
ax[1].set_xlabel('Angles (degrees)')
ax[1].set_ylabel('Distance (pixels)')
ax[1].axis('image')

ax[2].imshow(image, cmap=cm.gray)
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_axis_off()
ax[2].set_title('Detected lines')

for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
    ax[2].axline((x0, y0), slope=np.tan(angle + np.pi/2))

plt.tight_layout()
plt.show()

Рассмотрим применение вероятностного преобразования Хафа

In [ ]:
# Line finding using the Probabilistic Hough Transform
image = data.camera()
edges = canny(image, 2, 1, 25)
lines = probabilistic_hough_line(edges, threshold=10, line_length=5,
                                 line_gap=3)

In [ ]:
# Generating figure 2
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(edges, cmap=cm.gray)
ax[1].set_title('Canny edges')

ax[2].imshow(edges * 0)
for line in lines:
    p0, p1 = line
    ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[2].set_xlim((0, image.shape[1]))
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_title('Probabilistic Hough')

for a in ax:
    a.set_axis_off()

plt.tight_layout()
plt.show()


## 3.2. Метод морфологической обработки контуров Morphological Snakes
Morphological Snakes - это семейство методов сегментации изображений. Их поведение аналогично поведению методов на основе активных контуров (например, «Геодезические активные контуры» или «Активные контуры без ребер»). Однако Morphological Snakes используют морфологические операторы (такие как дилатация или эрозия). Это делает данное семейство методов быстрее и численно более стабильными, чем их традиционные аналоги.

В реализации Scikit-image доступны два метода Morphological Snakes: морфологические геодезические активные контуры (MorphGAC, реализованные в функции morphological_geodesic_active_contour) и морфологические активные контуры без краев (MorphACWE, реализованные в функции morphological_chan_vese).

MorphGAC подходит для изображений с видимыми контурами, даже если эти контуры могут быть зашумленными или частично нечеткими. Однако требуется предварительная обработка изображения для выделения контуров. Это можно сделать с помощью функции inverse_gaussian_gradient. Качество сегментации MorphGAC во многом зависит от этого шага предварительной обработки.

Напротив, MorphACWE хорошо работает, когда значения пикселей внутренней и внешней областей объекта для сегментации имеют разные средние значения. В отличие от MorphGAC, MorphACWE не требует четкого определения контуров объекта и работает с исходным изображением без какой-либо предварительной обработки. Это упрощает использование и настройку MorphACWE.

In [ ]:
def store_evolution_in(lst):
    """Returns a callback function to store the evolution of the level sets in
    the given list.
    """

    def _store(x):
        lst.append(np.copy(x))

    return _store

In [ ]:
# Morphological ACWE
image = img_as_float(data.camera())

# Initial level set
init_ls = checkerboard_level_set(image.shape, 6)
# List with intermediate results for plotting the evolution
evolution = []
callback = store_evolution_in(evolution)
ls = morphological_chan_vese(image, 35, init_level_set=init_ls, smoothing=3,
                             iter_callback=callback)



fig, axes = plt.subplots(2, 2, figsize=(8, 8))
ax = axes.flatten()

ax[0].imshow(image, cmap="gray")
ax[0].set_axis_off()
ax[0].contour(ls, [0.5], colors='r')
ax[0].set_title("Morphological ACWE segmentation", fontsize=12)

ax[1].imshow(ls, cmap="gray")
ax[1].set_axis_off()
contour = ax[1].contour(evolution[2], [0.5], colors='g')
contour.collections[0].set_label("Iteration 2")
contour = ax[1].contour(evolution[7], [0.5], colors='y')
contour.collections[0].set_label("Iteration 7")
contour = ax[1].contour(evolution[-1], [0.5], colors='r')
contour.collections[0].set_label("Iteration 35")
ax[1].legend(loc="upper right")
title = "Morphological ACWE evolution"
ax[1].set_title(title, fontsize=12)



# Morphological GAC
image = img_as_float(data.coins())
gimage = inverse_gaussian_gradient(image)

# Initial level set
init_ls = np.zeros(image.shape, dtype=np.int8)
init_ls[10:-10, 10:-10] = 1
# List with intermediate results for plotting the evolution
evolution = []
callback = store_evolution_in(evolution)
ls = morphological_geodesic_active_contour(gimage, 230, init_ls,
                                           smoothing=1, balloon=-1,
                                           threshold=0.69,
                                           iter_callback=callback)

ax[2].imshow(image, cmap="gray")
ax[2].set_axis_off()
ax[2].contour(ls, [0.5], colors='r')
ax[2].set_title("Morphological GAC segmentation", fontsize=12)

ax[3].imshow(ls, cmap="gray")
ax[3].set_axis_off()
contour = ax[3].contour(evolution[0], [0.5], colors='g')
contour.collections[0].set_label("Iteration 0")
contour = ax[3].contour(evolution[100], [0.5], colors='y')
contour.collections[0].set_label("Iteration 100")
contour = ax[3].contour(evolution[-1], [0.5], colors='r')
contour.collections[0].set_label("Iteration 230")
ax[3].legend(loc="upper right")
title = "Morphological GAC evolution"
ax[3].set_title(title, fontsize=12)

fig.tight_layout()
plt.show()

## 4. Сегментирование на основе областей
## 4.1. Базовый алгоритм водоразделов
Водораздел - это классический алгоритм, используемый для сегментации, то есть для разделения различных объектов на изображении.

Начиная с определяемых пользователем маркеров, алгоритм водораздела обрабатывает значения пикселей как местную топографию (высоту). Алгоритм затопляет бассейны от маркеров до тех пор, пока бассейны, относящиеся к разным маркерам, не встретятся на линиях водоразделов. Во многих случаях маркеры выбираются в качестве локальных минимумов изображения.

В приведенном ниже примере необходимо разделить два перекрывающихся круга. Для этого вычисляется изображение, представляющее собой расстояние до фона. Максимумы этого расстояния выбраны в качестве маркеров, и затопление бассейнов от таких маркеров разделяет два круга вдоль линии водораздела.

In [ ]:
# Generate an initial image with two overlapping circles
x, y = np.indices((80, 80))
x1, y1, x2, y2 = 28, 28, 44, 52
r1, r2 = 16, 20
mask_circle1 = (x - x1)**2 + (y - y1)**2 < r1**2
mask_circle2 = (x - x2)**2 + (y - y2)**2 < r2**2
image = np.logical_or(mask_circle1, mask_circle2)

In [ ]:
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt(image)
coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=image)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)


labels = watershed(-distance, markers, mask=image)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[1].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

## 4.2. Поиск регулярных структур с помощью алгоритма компактного водораздела
Преобразование водораздела обычно используется в качестве отправной точки для многих алгоритмов сегментации. Однако без разумного выбора маркера начала обработки алгоритм может давать очень неравномерные размеры фрагментов, с которыми может быть трудно справиться при последующем анализе.

Оба алгоритма реализованы в функции skimage.morphology.watershed(). Чтобы использовать компактную форму, требуется передать в аргумент значение компактности больше 0.

In [ ]:
coins = data.coins()
edges = filters.sobel(coins)

grid = util.regular_grid(coins.shape, n_points=468)


seeds = np.zeros(coins.shape, dtype=int)
seeds[grid] = np.arange(seeds[grid].size).reshape(seeds[grid].shape) + 1


w0 = watershed(edges, seeds)
w1 = watershed(edges, seeds, compactness=0.01)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5))

ax0.imshow(color.label2rgb(w0, coins, bg_label=-1))
ax0.set_title('Classical watershed')

ax1.imshow(color.label2rgb(w1, coins, bg_label=-1))
ax1.set_title('Compact watershed')

plt.show()

## 4.3. Создание расширенной области вокруг результатов сегментации по водоразделам
При условии наличия нескольких связанных компонентов, представленных разметкой объектов, возможно расширить область локализации с помощью skimage.segmentation.expand_labels(). В отличие от skimage.morphology.dilation() этот метод не позволяет соединенным компонентам расширяться на соседние связанные компоненты с меньшим номером метки.

In [ ]:
coins = data.coins()

# Make segmentation using edge-detection and watershed.
edges = sobel(coins)

# Identify some background and foreground pixels from the intensity values.
# These pixels are used as seeds for watershed.
markers = np.zeros_like(coins)
foreground, background = 1, 2
markers[coins < 30.0] = background
markers[coins > 150.0] = foreground

ws = watershed(edges, markers)
seg1 = label(ws == foreground)


expanded = expand_labels(seg1, distance=10)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 5),
                         sharex=True, sharey=True)

color1 = label2rgb(seg1, image=coins, bg_label=0)
axes[0].imshow(color1)
axes[0].set_title('Sobel+Watershed')

color2 = label2rgb(expanded, image=coins, bg_label=0)
axes[1].imshow(color2)
axes[1].set_title('Expanded labels')

for a in axes:
    a.axis('off')
fig.tight_layout()
plt.show()

## 4.4. Поиск маркеров для сегментации с помощью водоразделов
Функция peak_local_max возвращает координаты локальных пиков (максимумов) изображения. Внутренне фильтр максимума используется для поиска локальных максимумов. Эта операция расширяет исходное изображение с помощью дилатации и объединяет соседние локальные максимумы, находящиеся в пределах дилатации. Места, где исходное изображение равно расширенному изображению, возвращаются как локальные максимумы.

In [ ]:
im = img_as_float(data.coins())

# image_max is the dilation of im with a 20*20 structuring element
# It is used within peak_local_max function
image_max = ndi.maximum_filter(im, size=20, mode='constant')

# Comparison between image_max and im to find the coordinates of local maxima
coordinates = peak_local_max(im, min_distance=20)

In [ ]:
# display results
fig, axes = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(im, cmap=plt.cm.gray)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(image_max, cmap=plt.cm.gray)
ax[1].axis('off')
ax[1].set_title('Maximum filter')

ax[2].imshow(im, cmap=plt.cm.gray)
ax[2].autoscale(False)
ax[2].plot(coordinates[:, 1], coordinates[:, 0], 'r.')
ax[2].axis('off')
ax[2].set_title('Peak local max')

fig.tight_layout()

plt.show()

## 4.5. Использование градиентов для получения маркеров преобразования водораздела
Получение маркера может быть выполнено на основании областей с низким значением градиента. В градиентном изображении области с высокими значениями соответствуют граничным областям. Использование в качестве маркеров более низких значений гарантирует, что сегментированные объекты будут найдены.

In [ ]:
image = img_as_ubyte(data.eagle())

# denoise image
denoised = rank.median(image, disk(2))

# find continuous region (low gradient -
# where less than 10 for this image) --> markers
# disk(5) is used here to get a more smooth image
markers = rank.gradient(denoised, disk(5)) < 10


markers = ndi.label(markers)[0]


# local gradient (disk(2) is used to keep edges thin)
gradient = rank.gradient(denoised, disk(2))

# process the watershed
labels = watershed(gradient, markers)

In [ ]:
# display results
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 8),
                         sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title("Original")

ax[1].imshow(gradient, cmap=plt.cm.nipy_spectral)
ax[1].set_title("Local Gradient")

ax[2].imshow(markers, cmap=plt.cm.nipy_spectral)
ax[2].set_title("Markers")

ax[3].imshow(image, cmap=plt.cm.gray)
ax[3].imshow(labels, cmap=plt.cm.nipy_spectral, alpha=.5)
ax[3].set_title("Segmented")

for a in ax:
    a.axis('off')

fig.tight_layout()
plt.show()

## 4.6. Метод заливки
Заливка - это алгоритм для идентификации и / или изменения соседних значений в изображении на основе их сходства с исходной точкой.

Рассмотрим простейший пример выполнения заливки

In [ ]:
checkers = data.checkerboard()

# Fill a square near the middle with value 127, starting at index (76, 76)
filled_checkers = flood_fill(checkers, (76, 76), 127)

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))

ax[0].imshow(checkers, cmap=plt.cm.gray)
ax[0].set_title('Original')

ax[1].imshow(filled_checkers, cmap=plt.cm.gray)
ax[1].plot(76, 76, 'wo')  # seed point
ax[1].set_title('After flood fill')

plt.show()

Поскольку стандартная заливка требует, чтобы соседние пиксели были строго одинаковыми, ее использование ограничено в реальных изображениях с цветовыми градиентами и шумом. Для того, чтобы задавать диапазон возможных значений соседних пикселей указывается специальный аргумент. 

In [ ]:
cameraman = data.camera()

# Change the cameraman's coat from dark to light (255).  The seed point is
# chosen as (155, 150)
light_coat = flood_fill(cameraman, (155, 150), 255, tolerance=10)
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))

ax[0].imshow(cameraman, cmap=plt.cm.gray)
ax[0].set_title('Original')
ax[0].axis('off')

ax[1].imshow(light_coat, cmap=plt.cm.gray)
ax[1].plot(150, 155, 'ro')  # seed point
ax[1].set_title('After flood fill')
ax[1].axis('off')

plt.show()

Проварьируем диапазон возможных допустимых значений соседних пикселей.

In [ ]:
output = []

for i in range(8):
    tol = 5 + 20 * i
    output.append(flood_fill(cameraman, (0, 0), 255, tolerance=tol))

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))
ax[0, 0].imshow(cameraman, cmap=plt.cm.gray)
ax[0, 0].set_title('Original')
ax[0, 0].axis('off')

# Plot all eight different tolerances for comparison.
for i in range(8):
    m, n = np.unravel_index(i + 1, (3, 3))
    ax[m, n].imshow(output[i], cmap=plt.cm.gray)
    ax[m, n].set_title('Tolerance {0}'.format(str(5 + 20 * i)))
    ax[m, n].axis('off')
    ax[m, n].plot(0, 0, 'bo')  # seed point

fig.tight_layout()
plt.show()

Доступна родственная функция flood, которая возвращает маску, получаемую по принципу заливки, исходное изображение не изменяется. Это полезно для целей сегментации и более сложных конвейеров анализа.

In [ ]:
cat = data.chelsea()
cat_sobel = filters.sobel(cat[..., 0])
cat_nose = flood(cat_sobel, (240, 265), tolerance=0.03)

In [ ]:
fig, ax = plt.subplots(nrows=3, figsize=(10, 10))

ax[0].imshow(cat)
ax[0].set_title('Original')
ax[0].axis('off')

ax[1].imshow(cat_sobel)
ax[1].set_title('Sobel filtered')
ax[1].axis('off')

ax[2].imshow(cat)
ax[2].imshow(cat_nose, cmap=plt.cm.gray, alpha=0.3)
ax[2].plot(265, 240, 'wo')  # seed point
ax[2].set_title('Nose segmented with `flood`')
ax[2].axis('off')

fig.tight_layout()
plt.show()

Поскольку заливка применима к одноканальным изображениям, можно преобразовать изображение в пространство HSV, чтобы заливать пиксели аналогичного оттенка.

In [ ]:
img = data.astronaut()

img_hsv = color.rgb2hsv(img)
img_hsv_copy = np.copy(img_hsv)

# flood function returns a mask of flooded pixels
mask = flood(img_hsv[..., 0], (313, 160), tolerance=0.016)

# Set pixels of mask to new value for hue channel
img_hsv[mask, 0] = 0.5

# Post-processing in order to improve the result
# Remove white pixels from flag, using saturation channel
mask_postprocessed = np.logical_and(mask,
                                    img_hsv_copy[..., 1] > 0.4)

# Remove thin structures with binary opening
mask_postprocessed = morphology.binary_opening(mask_postprocessed,
                                               np.ones((3, 3)))

# Fill small holes with binary closing
mask_postprocessed = morphology.binary_closing(
                mask_postprocessed, morphology.disk(20))
img_hsv_copy[mask_postprocessed, 0] = 0.5

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(color.hsv2rgb(img_hsv))
ax[0].axis('off')
ax[0].set_title('After flood fill')
ax[1].imshow(color.hsv2rgb(img_hsv_copy))
ax[1].axis('off')
ax[1].set_title('After flood fill and post-processing')

fig.tight_layout()
plt.show()

## 5. Методы сегментации на основе построения суперпикселей
Суперпиксель можно определить как группу пикселей, которые имеют общие характеристики (например, интенсивность пикселей). Суперпиксели становятся полезными во многих алгоритмах компьютерного зрения и обработки изображений, таких как сегментация изображения, семантическая маркировка, обнаружение и отслеживание объектов по следующим причинам:
- Они несут больше информации, чем пиксели.
- Суперпиксели имеют перцепционное значение, поскольку пиксели, принадлежащие данному суперпикселю, обладают схожими визуальными свойствами.
- Они обеспечивают удобное и компактное представление изображений, которое может быть очень полезно для задач, требующих большого объема вычислений.

## 5.1. Метод SLIС
SLIC (Простая линейная итеративная кластеризация) является алгоритмом для генерации суперпикселей. Этот алгоритм генерирует суперпиксели путем кластеризации пикселей на основе их цветового сходства и близости в плоскости изображения. Это делается в пятимерном пространстве [labxy], где [lab] - это цветовой вектор пикселя в цветовом пространстве CIELAB, а xy - позиция пикселя.

In [ ]:
img = data.coffee()



labels1 = segmentation.slic(img, compactness=30, n_segments=400, start_label=1)
out1 = color.label2rgb(labels1, img, kind='avg', bg_label=0)
out1 = out1.astype(np.uint8)

plt.imshow(out1)
plt.axis("off")
plt.show()

## 5.2. Метод maskSLIC
Метод maskSLIC является расширением метода SLIC для генерации суперпикселей в конкретной интересующей области. maskSLIC может решить проблемы с границами, которые влияют на метод SLIC.

Чтобы проиллюстрировать эти методы сегментации, воспользуемся изображением биологической ткани с иммуногистохимическим (ИГХ) окрашиванием.

In [ ]:
# Input data
img = data.immunohistochemistry()

# Compute a mask
lum = color.rgb2gray(img)
mask = morphology.remove_small_holes(
    morphology.remove_small_objects(
        lum < 0.7, 500),
    500)

mask = morphology.opening(mask, morphology.disk(3))

# SLIC result
slic = segmentation.slic(img, n_segments=200, start_label=1)

# maskSLIC result
m_slic = segmentation.slic(img, n_segments=100, mask=mask, start_label=1)

In [ ]:
# Display result
fig, ax_arr = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10, 10))
ax1, ax2, ax3, ax4 = ax_arr.ravel()

ax1.imshow(img)
ax1.set_title('Original image')

ax2.imshow(mask, cmap='gray')
ax2.set_title('Mask')

ax3.imshow(segmentation.mark_boundaries(img, slic))
ax3.contour(mask, colors='red', linewidths=1)
ax3.set_title('SLIC')

ax4.imshow(segmentation.mark_boundaries(img, m_slic))
ax4.contour(mask, colors='red', linewidths=1)
ax4.set_title('maskSLIC')

for ax in ax_arr.ravel():
    ax.set_axis_off()

plt.tight_layout()
plt.show()

## 6. Методы сегментации на основе графов (Region Adjacency Graphs)
Пример ниже демонстрирует использование функции merge_nodes графа смежности регионов (RAG). Класс RAG представляет собой неориентированный взвешенный граф, наследуемый от класса networkx.graph. Когда новый узел формируется путем слияния двух узлов, вес всех ребер, попадающих в результирующий узел, может быть обновлен с помощью определяемой пользователем функции weight_func.

По умолчанию в случае конфликта используется меньший вес ребра. В приведенном ниже примере также показано, как использовать пользовательскую функцию для выбора большего веса в графе.

In [ ]:
def max_edge(g, src, dst, n):
    """Callback to handle merging nodes by choosing maximum weight.

    Returns a dictionary with `"weight"` set as either the weight between
    (`src`, `n`) or (`dst`, `n`) in `g` or the maximum of the two when
    both exist.

    Parameters
    ----------
    g : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `g` to be merged.
    n : int
        A neighbor of `src` or `dst` or both.

    Returns
    -------
    data : dict
        A dict with the "weight" attribute set the weight between
        (`src`, `n`) or (`dst`, `n`) in `g` or the maximum of the two when
        both exist.
    """

    w1 = g[n].get(src, {'weight': -np.inf})['weight']
    w2 = g[n].get(dst, {'weight': -np.inf})['weight']
    return {'weight': max(w1, w2)}

In [ ]:
def display(g, title):
    """Displays a graph with the given title."""
    pos = nx.circular_layout(g)
    plt.figure()
    plt.title(title)
    nx.draw(g, pos)
    nx.draw_networkx_edge_labels(g, pos, font_size=20)

In [ ]:
g = RAG()
g.add_edge(1, 2, weight=10)
g.add_edge(2, 3, weight=20)
g.add_edge(3, 4, weight=30)
g.add_edge(4, 1, weight=40)
g.add_edge(1, 3, weight=50)

# Assigning dummy labels.
for n in g.nodes():
    g.nodes[n]['labels'] = [n]

gc = g.copy()
display(g, "Original Graph")
plt.show()

In [ ]:
g.merge_nodes(1, 3)
display(g, "Merged with default (min)")
plt.show()

In [ ]:
gc.merge_nodes(1, 3, weight_func=max_edge, in_place=False)
display(gc, "Merged with max without in_place")
plt.show()

## 6.1. Применение порога на основе графовой модели RAG
В этом примере создается граф смежности регионов (RAG) и объединяются области, похожие по цвету. В ходе операции объединяются области с похожим средним уровнем цвета.

In [ ]:
img = data.coffee()

g = graph.rag_mean_color(img, labels1)
labels2 = graph.cut_threshold(labels1, g, 29)
out2 = color.label2rgb(labels2, img, kind='avg', bg_label=0)
out2 = out2.astype(np.uint8)

fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True,
                       figsize=(6, 8))

ax[0].imshow(img)
ax[1].imshow(out2)

for a in ax:
    a.axis('off')

plt.tight_layout()

## 6.2. Выделение границ областей на основе графовых моделей RAG
Для определения границ областей с помощью графов смежности регионов используется функция rag_boundary. Функция skimage.future.graph.rag_boundary() принимает аргумент edge_map, который дает значение функции (например, краев).

In [ ]:
img = data.coffee()
gimg = color.rgb2gray(img)

labels = segmentation.slic(img, compactness=30, n_segments=400, start_label=1)
edges = filters.sobel(gimg)
edges_rgb = color.gray2rgb(edges)

g = graph.rag_boundary(labels, edges)
lc = graph.show_rag(labels, g, edges_rgb, img_cmap=None, edge_cmap='viridis',
                    edge_width=1.2)

plt.colorbar(lc, fraction=0.03)
plt.show()

Для визуальной оценки качества сегментации областей с помощью RAG можно одновременно визуализировать графовые структуры на исходном изображении.

In [ ]:
img = data.coffee()
labels = segmentation.slic(img, compactness=30, n_segments=400, start_label=1)
g = graph.rag_mean_color(img, labels)

fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))

ax[0].set_title('RAG drawn with default settings')
lc = graph.show_rag(labels, g, img, ax=ax[0])
# specify the fraction of the plot area that will be used to draw the colorbar
fig.colorbar(lc, fraction=0.03, ax=ax[0])

ax[1].set_title('RAG drawn with grayscale image and viridis colormap')
lc = graph.show_rag(labels, g, img,
                    img_cmap='gray', edge_cmap='viridis', ax=ax[1])
fig.colorbar(lc, fraction=0.03, ax=ax[1])

for a in ax:
    a.axis('off')

plt.tight_layout()
plt.show()

## 7. Алгоритм сегментации на основе интерактивных маркеров
Алгоритм случайного прохождения (Random Walker Algorithm) - это алгоритм сегментации изображения, при котором пользователь может интерактивно промаркировать небольшое количество пикселей известными метками, например, «объект» и «фон».

Предполагается, что каждый немаркированный пиксель высвобождает случайного блуждающего, и вычисляется вероятность того, что случайный блуждающий элемент каждого пикселя сначала достигает начального числа с каждой меткой, т. е. если пользователь помещает K начальных чисел, каждый с другой меткой, тогда необходимо вычислить для каждого пикселя вероятность того, что случайный элемент, покинувший пиксель, первым достигнет каждого начального числа. Эти вероятности можно определить аналитически, решив систему линейных уравнений. После вычисления этих вероятностей для каждого пикселя пикселю присваивается метка, для которой он, скорее всего, отправит случайного блуждающего. Изображение моделируется как граф, в котором каждый пиксель соответствует узлу, который соединен с соседними пикселями краями, а края взвешиваются, чтобы отразить сходство между пикселями. Следовательно, случайное блуждание происходит на взвешенном графе.

In [ ]:
# Generate noisy synthetic data
data_1 = skimage.img_as_float(binary_blobs(length=128, rng=1))
sigma = 0.35
data_1 += np.random.normal(loc=0, scale=sigma, size=data_1.shape)
data_1 = rescale_intensity(data_1, in_range=(-sigma, 1 + sigma),
                         out_range=(-1, 1))

# The range of the binary image spans over (-1, 1).
# We choose the hottest and the coldest pixels as markers.
markers = np.zeros(data_1.shape, dtype=np.uint)
markers[data_1 < -0.95] = 1
markers[data_1 > 0.95] = 2

# Run random walker algorithm
labels = random_walker(data_1, markers, beta=10, mode='bf')

In [ ]:
# Plot results
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 3.2),
                                    sharex=True, sharey=True)
ax1.imshow(data_1, cmap='gray')
ax1.axis('off')
ax1.set_title('Noisy data')
ax2.imshow(markers, cmap='magma')
ax2.axis('off')
ax2.set_title('Markers')
ax3.imshow(labels, cmap='gray')
ax3.axis('off')
ax3.set_title('Segmentation')

fig.tight_layout()
plt.show()

## 8. Метод сегментации на основе извлечения признаков
## 8.1. Извлечение морфометрических признаков объектов

In [ ]:
image = np.zeros((600, 600))

rr, cc = ellipse(300, 350, 100, 220)
image[rr, cc] = 1

image = rotate(image, angle=15, order=0)

rr, cc = ellipse(100, 100, 60, 50)
image[rr, cc] = 1

label_img = label(image)
regions = regionprops(label_img)

plt.imshow(label_img)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image, cmap=plt.cm.gray)

for props in regions:
    y0, x0 = props.centroid
    orientation = props.orientation
    x1 = x0 + math.cos(orientation) * 0.5 * props.minor_axis_length
    y1 = y0 - math.sin(orientation) * 0.5 * props.minor_axis_length
    x2 = x0 - math.sin(orientation) * 0.5 * props.major_axis_length
    y2 = y0 - math.cos(orientation) * 0.5 * props.major_axis_length

    ax.plot((x0, x1), (y0, y1), '-r', linewidth=2.5)
    ax.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
    ax.plot(x0, y0, '.g', markersize=15)

    minr, minc, maxr, maxc = props.bbox
    bx = (minc, maxc, maxc, minc, minc)
    by = (minr, minr, maxr, maxr, minr)
    ax.plot(bx, by, '-b', linewidth=2.5)

ax.axis((0, 600, 600, 0))
plt.show()

Мы используем skimage.measure.regionprops_table() для вычисления (выбранных) свойств для каждого региона

In [ ]:
props = regionprops_table(label_img, properties=('centroid',
                                                 'orientation',
                                                 'major_axis_length',
                                                 'minor_axis_length'))
pd.DataFrame(props)

## 8.2. Метод случайного леса для сегментации на основе локальных признаков random forests
Сегментация на основе обучения вычисляется с использованием локальных функций на основе локальной интенсивности, краев и текстур в разных масштабах. Предоставляемая пользователем маска используется для идентификации различных областей. Пиксели маски используются для обучения классификатора случайного леса из scikit-learn. Затем неразмеченные пиксели маркируются на основе прогноза классификатора.

In [ ]:
full_img = data.skin()

plt.imshow(full_img)
plt.show()

img = full_img[:900, :900]

# Build an array of labels for training the segmentation.
# Here we use rectangles but visualization libraries such as plotly
# can be used to draw a mask on the image.
training_labels = np.zeros(img.shape[:2], dtype=np.uint8)
training_labels[:130] = 1
training_labels[:170, :400] = 1
training_labels[600:900, 200:650] = 2
training_labels[330:430, 210:320] = 3
training_labels[260:340, 60:170] = 4
training_labels[150:200, 720:860] = 4

In [ ]:
sigma_min = 1
sigma_max = 16
features_func = partial(feature.multiscale_basic_features,
                        intensity=True, edges=False, texture=True,
                        sigma_min=sigma_min, sigma_max=sigma_max,
                        channel_axis=-1)
features = features_func(img)

In [ ]:
features.shape

In [ ]:
plt.imshow(features[:,:,4], cmap="gray")

In [ ]:
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1,
                             max_depth=10, max_samples=0.05)
clf = future.fit_segmenter(training_labels, features, clf)
result = future.predict_segmenter(features, clf)

In [ ]:
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(9, 10))
ax[0].imshow(segmentation.mark_boundaries(img, result, mode='thick'))
ax[0].contour(training_labels)
ax[0].set_title('Image, mask and segmentation boundaries')
ax[1].imshow(result)
ax[1].set_title('Segmentation')
fig.tight_layout()

## 8.3. Оценка важности признаков
Ниже мы рассмотрим важность различных функций, рассчитанную с помощью scikit-learn. Характеристики интенсивности имеют гораздо большее значение, чем особенности текстуры. Может возникнуть соблазн использовать эту информацию для уменьшения количества признаков, предоставляемых классификатору, чтобы сократить время вычислений. Однако это может привести к переобучению и ухудшению результатов на границе между областями.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
l = len(clf.feature_importances_)

feature_importance = (
        clf.feature_importances_[:l//3],
        clf.feature_importances_[l//3:2*l//3],
        clf.feature_importances_[2*l//3:])

sigmas = np.logspace(
        np.log2(sigma_min), np.log2(sigma_max),
        num=int(np.log2(sigma_max) - np.log2(sigma_min) + 1),
        base=2, endpoint=True)

for ch, color in zip(range(3), ['r', 'g', 'b']):
    ax[0].plot(sigmas, feature_importance[ch][::3], 'o', color=color)
    ax[0].set_title("Intensity features")
    ax[0].set_xlabel("$\\sigma$")

for ch, color in zip(range(3), ['r', 'g', 'b']):
    ax[1].plot(sigmas, feature_importance[ch][1::3], 'o', color=color)
    ax[1].plot(sigmas, feature_importance[ch][2::3], 's', color=color)
    ax[1].set_title("Texture features")
    ax[1].set_xlabel("$\\sigma$")

fig.tight_layout()

## 8.4. Применение обученной модели к новому изображению
Далее можно применить обученный классификатор к новым объектам

In [ ]:
img_new = full_img[:700, 900:]

plt.imshow(img_new)
plt.show()


features_new = features_func(img_new)
result_new = future.predict_segmenter(features_new, clf)


fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(6, 10))
ax[0].imshow(segmentation.mark_boundaries(img_new, result_new, mode='thick'))
ax[0].set_title('Image')
ax[1].imshow(result_new)
ax[1].set_title('Segmentation')
fig.tight_layout()

plt.show()

## 9. Оценка качества выполнения сегментации
Для эмпирического определения наиболее подходящего алгоритма сегментации можно использовать различные метрики. Мы будем использовать адаптированную ошибку Рэнда и вариацию информации в качестве примера показателей и посмотрим, как избыточная сегментация (разделение истинных сегментов на слишком много подсегментов) и недостаточная сегментация (объединение разных истинных сегментов в один сегмент) влияют на разные оценки.

In [ ]:
image = data.coins()

Во-первых, мы производим априори корректную сегментацию. Для этого простого изображения мы знаем точные функции и параметры, которые позволят произвести идеальную сегментацию.

In [ ]:
elevation_map = sobel(image)
markers = np.zeros_like(image)
markers[image < 30] = 1
markers[image > 150] = 2
im_true = watershed(elevation_map, markers)
im_true = ndi.label(ndi.binary_fill_holes(im_true - 1))[0]

Затем мы выполняем три разных сегментации с разными характеристиками. Первый подход использует skimage.segmentation.watershed() с компактностью, которая является полезной начальной сегментацией.

In [ ]:
edges = sobel(image)
im_test1 = watershed(edges, markers=468, compactness=0.001)

В следующем подходе используется краевой фильтр Кэнни, skimage.filters.canny(). Это очень хороший детектор краев, который дает сбалансированные результаты.

In [ ]:
edges = canny(image)
fill_coins = ndi.binary_fill_holes(edges)
im_test2 = ndi.label(remove_small_objects(fill_coins, 21))[0]

Наконец, мы используем морфологические геодезические активные контуры, skimage.segmentation.morphological_geodesic_active_contour(), метод, который обычно дает хорошие результаты, но требует много времени, чтобы прийти к хорошему ответу.

In [ ]:
image = img_as_float(image)
gradient = inverse_gaussian_gradient(image)
init_ls = np.zeros(image.shape, dtype=np.int8)
init_ls[10:-10, 10:-10] = 1
im_test3 = morphological_geodesic_active_contour(gradient, num_iter=100,
                                                 init_level_set=init_ls,
                                                 smoothing=1, balloon=-1,
                                                 threshold=0.69)
im_test3 = label(im_test3)

In [ ]:
method_names = ['Compact watershed', 'Canny filter',
                'Morphological Geodesic Active Contours']
short_method_names = ['Compact WS', 'Canny', 'GAC']

precision_list = []
recall_list = []
split_list = []
merge_list = []

for name, im_test in zip(method_names, [im_test1, im_test2, im_test3]):
    error, precision, recall = adapted_rand_error(im_true, im_test)
    splits, merges = variation_of_information(im_true, im_test)
    split_list.append(splits)
    merge_list.append(merges)
    precision_list.append(precision)
    recall_list.append(recall)
    print(f"\n## Method: {name}")
    print(f"Adapted Rand error: {error}")
    print(f"Adapted Rand precision: {precision}")
    print(f"Adapted Rand recall: {recall}")
    print(f"False Splits: {splits}")
    print(f"False Merges: {merges}")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 6), constrained_layout=True)
ax = axes.ravel()

ax[0].scatter(merge_list, split_list)
for i, txt in enumerate(short_method_names):
    ax[0].annotate(txt, (merge_list[i], split_list[i]),
                   verticalalignment='center')
ax[0].set_xlabel('False Merges (bits)')
ax[0].set_ylabel('False Splits (bits)')
ax[0].set_title('Split Variation of Information')

ax[1].scatter(precision_list, recall_list)
for i, txt in enumerate(short_method_names):
    ax[1].annotate(txt, (precision_list[i], recall_list[i]),
                   verticalalignment='center')
ax[1].set_xlabel('Precision')
ax[1].set_ylabel('Recall')
ax[1].set_title('Adapted Rand precision vs. recall')
ax[1].set_xlim(0, 1)
ax[1].set_ylim(0, 1)

ax[2].imshow(mark_boundaries(image, im_true))
ax[2].set_title('True Segmentation')
ax[2].set_axis_off()

ax[3].imshow(mark_boundaries(image, im_test1))
ax[3].set_title('Compact Watershed')
ax[3].set_axis_off()

ax[4].imshow(mark_boundaries(image, im_test2))
ax[4].set_title('Edge Detection')
ax[4].set_axis_off()

ax[5].imshow(mark_boundaries(image, im_test3))
ax[5].set_title('Morphological GAC')
ax[5].set_axis_off()

plt.show()

Пример классического метода сегментации:
https://stackoverflow.com/questions/57813137/how-to-use-watershed-segmentation-in-opencv-python